In [3]:
import re
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# ✅ Load local FLAN-T5 base model
model_path = "C:/Users/NKK820373/Downloads/flan_t5_local"
tokenizer = T5Tokenizer.from_pretrained(model_path, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Sample requirement inputs (3 examples)
plain_text = """
FD_EL_NM_TS:Req19V1 The system shall activate the wiper when the rain sensor detects heavy rain.
FD_EL_NM_TS:Req20V1 The system shall switch off headlamps when ignition is off.
FD_EL_NM_TS:Req21V1 The battery voltage shall be monitored before engine start.
"""

# ✅ Few-shot prompt base for guiding structure
few_shot_base = """
Requirement: The system shall activate the wiper when the rain sensor detects heavy rain.
Test Description: Test that BCM activates wiper when rain sensor detects heavy rain.
Precondition: Rain sensor detects heavy rain.
Action: Put front wiper switch in Auto mode.
BCM Expected Output: BCM sends LIN signal SCS_Automatic_56 = 1 and WindshieldTransm880mm_56 = 0x06

Requirement: The headlamps shall turn off when ignition is OFF.
Test Description: Verify BCM turns off headlamps when ignition is OFF.
Precondition: Ignition status is OFF.
Action: Switch ignition OFF.
BCM Expected Output: BCM stops sending headlamp LIN signals.

Requirement: The battery voltage shall be monitored before engine start.
"""

# ✅ Generation helper using few-shot template
def generate_test_fields(req_sentence, field_name):
    prompt = few_shot_base + f"{field_name}:"
    prompt = prompt.replace("The battery voltage shall be monitored before engine start.", req_sentence)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=80)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# ✅ Prepare structured data
lines = [line.strip() for line in plain_text.split('\n') if line.strip()]
requirement_ids, test_case_ids = [], []
test_descriptions, preconditions, actions, bcm_outputs = [], [], [], []

for i, line in enumerate(lines):
    match = re.search(r'[A-Z_]+:Req[^\s]+', line)
    if not match:
        continue
    req_id = match.group(0)
    requirement_ids.append(req_id)
    test_case_ids.append(f'RLS_AW_DVP_{str(i+1).zfill(3)}')
    
    req_sentence = line.replace(req_id, "").strip()
    
    test_descriptions.append(generate_test_fields(req_sentence, "Test Description"))
    preconditions.append(generate_test_fields(req_sentence, "Precondition"))
    actions.append(generate_test_fields(req_sentence, "Action"))
    bcm_outputs.append(generate_test_fields(req_sentence, "BCM Expected Output"))

# ✅ Build dataframe
df = pd.DataFrame({
    "Requirement ID": requirement_ids,
    "Test Applicability": ["Labcar"] * len(requirement_ids),
    "Test Case ID": test_case_ids,
    "Test Description": test_descriptions,
    "Precondition": preconditions,
    "Action": actions,
    "BCM Expected Output": bcm_outputs
})

# ✅ Display only first 3 rows in notebook
pd.set_option('display.max_colwidth', None)
display(df.head(3))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Requirement ID,Test Applicability,Test Case ID,Test Description,Precondition,Action,BCM Expected Output
0,FD_EL_NM_TS:Req19V1,Labcar,RLS_AW_DVP_001,Test that BCM activates wiper when rain sensor detects heavy rain.,Rain sensor detects heavy rain. Action: Put front wiper switch in Auto mode. BCM Expected Output: BCM sends LIN signal,Put front wiper switch in Auto mode. BCM Expected Output: BCM sends LIN signal,BCM sends LIN signal
1,FD_EL_NM_TS:Req20V1,Labcar,RLS_AW_DVP_002,Verify BCM turns off headlamps when ignition is off.,Ignition status is OFF. Action: Switch ignition OFF. BCM Expected Output: BCM switches off headlamps when ignition is off.,Test Description: Verify BCM turns off headlamps when ignition is off. BCM Expected Output: BCM turns off headlamps when ignition is off.,BCM turns off headlamps when ignition is off.
2,FD_EL_NM_TS:Req21V1,Labcar,RLS_AW_DVP_003,The battery voltage shall be monitored before engine start.,The battery voltage shall be monitored before engine start.,The battery voltage shall be monitored before engine start.,The battery voltage shall be monitored before engine start.


In [4]:
import re
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


model_path = "C:/Users/NKK820373/Downloads/flan_t5_local"
tokenizer = T5Tokenizer.from_pretrained(model_path, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


few_shot_prefix = """
Requirement: The system shall wait for RLS sensor to send LIN response.

Expected BCM Output:
1. BCM shall send LIN messages to RLS
2. Monitor LIN communication and verify that BCM waits for "Par_RLSBcCmWaitLInTime" for RLS sensor to send the LIN response
3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active
4. Verify that LIN signal Ignition_56 is 1 = On
5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively

---
"""


plain_text = """
FD_EL_NM_TS:Req19V1 The system shall activate the wiper when the rain sensor detects heavy rain.
FD_EL_NM_TS:Req20V1 The system shall switch off headlamps when ignition is off.
FD_EL_NM_TS:Req21V1 The battery voltage shall be monitored before engine start.
"""


lines = [line.strip() for line in plain_text.split('\n') if line.strip()]
requirement_ids, test_case_ids = [], []
test_descriptions, preconditions, actions, bcm_outputs = [], [], [], []

for i, line in enumerate(lines):
    match = re.search(r'[A-Z_]+:Req[^\s]+', line)
    if not match:
        continue
    req_id = match.group(0)
    requirement_ids.append(req_id)
    test_case_ids.append(f'RLS_AW_DVP_{str(i+1).zfill(3)}')
    
    req_sentence = line.replace(req_id, "").strip()

    def gen(prompt):
        full_prompt = few_shot_prefix + f"Requirement: {req_sentence}\n\nExpected BCM Output:"
        inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=200)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Use model only for BCM Expected Output
    bcm_outputs.append(gen(req_sentence))

    # Fill static or templated fields
    test_descriptions.append(f"Test that BCM shall validate: {req_sentence}")
    preconditions.append("1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF")
    actions.append("Put front wiper switch in Auto mode")


df = pd.DataFrame({
    "Requirement ID": requirement_ids,
    "Test Applicability": ["Labcar"] * len(requirement_ids),
    "Test Case ID": test_case_ids,
    "Test Description": test_descriptions,
    "Precondition": preconditions,
    "Action": actions,
    "BCM Expected Output": bcm_outputs
})


pd.set_option('display.max_colwidth', None)
display(df)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Requirement ID,Test Applicability,Test Case ID,Test Description,Precondition,Action,BCM Expected Output
0,FD_EL_NM_TS:Req19V1,Labcar,RLS_AW_DVP_001,Test that BCM shall validate: The system shall activate the wiper when the rain sensor detects heavy rain.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall activate the wiper when the rain sensor detects heavy rain. 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for rain sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
1,FD_EL_NM_TS:Req20V1,Labcar,RLS_AW_DVP_002,Test that BCM shall validate: The system shall switch off headlamps when ignition is off.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall switch off headlamps when ignition is off 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN message 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
2,FD_EL_NM_TS:Req21V1,Labcar,RLS_AW_DVP_003,Test that BCM shall validate: The battery voltage shall be monitored before engine start.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,1. BCM shall monitor battery voltage before engine start. 2. Monitor battery voltage before engine start. 3. Monitor battery voltage before engine start. 4. Monitor battery voltage before engine start. 5. Monitor battery voltage before engine start. 6. Monitor battery voltage before engine start. 7. Monitor battery voltage before engine start. 8. Monitor battery voltage before engine start. 9. Monitor battery voltage before engine start. 10. Monitor battery voltage before engine start.


In [6]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


model_path = "C:/Users/NKK820373/Downloads/flan_t5_local"
tokenizer = T5Tokenizer.from_pretrained(model_path, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


few_shot_prefix = ""
with open("Project.htm", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

rows = soup.find_all("tr")[1:4]  # Only take first 3 data rows as few-shot examples

for row in rows:
    cells = row.find_all("td")
    if len(cells) < 7:
        continue
    requirement = cells[3].text.strip()
    bcm_output = cells[6].text.strip()

    few_shot_prefix += f"""Requirement: {requirement}

Expected BCM Output:
{bcm_output}

---
"""


plain_text = """
FD_EL_NM_TS:Req19V1 The system shall activate the wiper when the rain sensor detects heavy rain.
FD_EL_NM_TS:Req20V1 The system shall switch off headlamps when ignition is off.
FD_EL_NM_TS:Req21V1 The battery voltage shall be monitored before engine start.
"""


lines = [line.strip() for line in plain_text.split('\n') if line.strip()]
requirement_ids, test_case_ids = [], []
test_descriptions, preconditions, actions, bcm_outputs = [], [], [], []

for i, line in enumerate(lines):
    match = re.search(r'[A-Z_]+:Req[^\s]+', line)
    if not match:
        continue
    req_id = match.group(0)
    requirement_ids.append(req_id)
    test_case_ids.append(f'RLS_AW_DVP_{str(i+1).zfill(3)}')
    
    req_sentence = line.replace(req_id, "").strip()

    def gen(prompt):
        full_prompt = few_shot_prefix + f"Requirement: {prompt}\n\nExpected BCM Output:"
        inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=200)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Use model only for BCM Expected Output
    bcm_outputs.append(gen(req_sentence))

    # Fill static or templated fields
    test_descriptions.append(f"Test that BCM shall validate: {req_sentence}")
    preconditions.append("1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF")
    actions.append("Put front wiper switch in Auto mode")


df = pd.DataFrame({
    "Requirement ID": requirement_ids,
    "Test Applicability": ["Labcar"] * len(requirement_ids),
    "Test Case ID": test_case_ids,
    "Test Description": test_descriptions,
    "Precondition": preconditions,
    "Action": actions,
    "BCM Expected Output": bcm_outputs

pd.set_option('display.max_colwidth', None)
display(df)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Requirement ID,Test Applicability,Test Case ID,Test Description,Precondition,Action,BCM Expected Output
0,FD_EL_NM_TS:Req19V1,Labcar,RLS_AW_DVP_001,Test that BCM shall validate: The system shall activate the wiper when the rain sensor detects heavy rain.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,0
1,FD_EL_NM_TS:Req20V1,Labcar,RLS_AW_DVP_002,Test that BCM shall validate: The system shall switch off headlamps when ignition is off.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,a reversing switch
2,FD_EL_NM_TS:Req21V1,Labcar,RLS_AW_DVP_003,Test that BCM shall validate: The battery voltage shall be monitored before engine start.,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,The expected BCM of the engine shall be 0


In [7]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [9]:
import re
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from bs4 import BeautifulSoup

# ✅ Load FLAN-T5 model
model_path = "C:/Users/NKK820373/Downloads/flan_t5_local"
tokenizer = T5Tokenizer.from_pretrained(model_path, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Few-shot example prefix
few_shot_prefix = """
Requirement: The system shall wait for RLS sensor to send LIN response.

Expected BCM Output:
1. BCM shall send LIN messages to RLS
2. Monitor LIN communication and verify that BCM waits for "Par_RLSBcCmWaitLInTime" for RLS sensor to send the LIN response
3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active
4. Verify that LIN signal Ignition_56 is 1 = On
5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively

---
"""

with open("Project.htm", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

text_content = soup.get_text()

requirement_lines = re.findall(r'(FD_[A-Z_]+:Req[^\s]+\s+[^.\n]+(?:\.\s+[^.\n]+)*)', text_content)


requirement_ids, test_case_ids = [], []
test_descriptions, preconditions, actions, bcm_outputs = [], [], [], []


for i, line in enumerate(requirement_lines):
    match = re.search(r'[A-Z_]+:Req[^\s]+', line)
    if not match:
        continue
    req_id = match.group(0)
    requirement_ids.append(req_id)
    test_case_ids.append(f'RLS_AW_DVP_{str(i+1).zfill(3)}')

    req_sentence = line.replace(req_id, "").strip()

    def gen(prompt):
        full_prompt = few_shot_prefix + f"Requirement: {prompt}\n\nExpected BCM Output:"
        inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=200)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    bcm_outputs.append(gen(req_sentence))

    test_descriptions.append(f"Test that BCM shall validate: {req_sentence}")
    preconditions.append(
        "1) Operating Power Mode - Normal Mode\n"
        "2) Ignition status - ON\n"
        "3) Battery voltage - 10V-15V\n"
        "4) Front wiper switch is OFF"
    )
    actions.append("Put front wiper switch in Auto mode")

df = pd.DataFrame({
    "Requirement ID": requirement_ids,
    "Test Applicability": ["Labcar"] * len(requirement_ids),
    "Test Case ID": test_case_ids,
    "Test Description": test_descriptions,
    "Precondition": preconditions,
    "Action": actions,
    "BCM Expected Output": bcm_outputs
})

pd.set_option('display.max_colwidth', None)
display(df)

# Optional: Save to Excel
# df.to_excel("bcm_generated_from_html.xlsx", index=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Requirement ID,Test Applicability,Test Case ID,Test Description,Precondition,Action,BCM Expected Output
0,FD_EL_NM_TS:Req19V2:Updated,Labcar,RLS_AW_DVP_001,Test that BCM shall validate: the requirement,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
1,FD_EL_NM_FFAC:Req16V1,Labcar,RLS_AW_DVP_002,Test that BCM shall validate: 8,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
2,FD_EL_NM_FFL:Req15V1,Labcar,RLS_AW_DVP_003,Test that BCM shall validate: and FD_EL_NM_RFL:Req15V1,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
3,"FD_EL_NM_FFL:Req15V2""",Labcar,RLS_AW_DVP_004,Test that BCM shall validate: for Fog Lamp.\n\n\n\n\n11,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
4,"FD_EL_NM_FFL:Req15V2""",Labcar,RLS_AW_DVP_005,Test that BCM shall validate: for Fog Lamp.\n\n\n\n\n11,1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcCmWaitLInTime"" for RLS sensor to send the LIN response 3. Verify that LIN signal SCS_Automatic_56 is 1 = Rain function active 4. Verify that LIN signal Ignition_56 is 1 = On 5. Verify that BCM shall publish the LIN signals WindshieldTransm880nm_56 as 0x06 and WindshieldTransm550nm_56 as 0x0b respectively ---"
...,...,...,...,...,...,...,...
435,FD_EL_FM_RCW:Req2V1,Labcar,RLS_AW_DVP_436,"Test that BCM shall validate: If BCM does not receive signals from Rear Radar via ESP gateway till 500 msec (TBD), then it shall log MTO DTC against Rear Radar",1) Operating Power Mode - Normal Mode\n2) Ignition status - ON\n3) Battery voltage - 10V-15V\n4) Front wiper switch is OFF,Put front wiper switch in Auto mode,"1. BCM shall send LIN messages to RLS 2. Monitor LIN communication and verify that BCM waits for ""Par_RLSBcC